In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def scrape_player_data(st_year, end_year, type):
    player_data = []
    for year in range(st_year, end_year):
        url = f'https://sports.ndtv.com/ipl-{year}/auction/{type}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        teams = ['CSK', 'DC', 'MI',
                'LSG', 'GT', 'KKR',
                'PBKS', 'RR', 'RCB',
                'SRH']

        for team in teams:
            table = soup.find('table', id=re.compile(team))
            if table:
                rows = table.find_all('tr')[1:]  # Skip header
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) >= 3:
                        player_name = player_name = cells[1].get_text(strip=True).replace(cells[2].get_text(strip=True), '').strip()
                        role = cells[2].get_text(strip=True)
                        price = cells[3].get_text(strip=True)
                        spans = cells[1].find_all('span')
                        Origin = 'Unknown'
                        for span in spans:
                            class_attr = span.get('class', [])
                            if 'vj-sp_india-flag' in class_attr:
                                Origin = 'Indian'
                                break
                            if 'vj-sp_airoplane'in class_attr:
                                Origin = 'Overseas'
                                break
                        if type == 'retainedplayer':
                            player_data.append({
                            'Player': player_name,
                            'Role': role,
                            'Price': price,
                            'Team': team,
                            'Year': year,
                            'Origin': Origin,
                            "Retained": 'Yes'})
                        else:
                            player_data.append({
                            'Player': player_name,
                            'Role': role,
                            'Price': price,
                            'Team': team,
                            'Year': year,
                            'Origin': Origin,
                        })
    return pd.DataFrame(player_data)


In [ ]:
retained_df = scrape_player_data(2022,2026,'retainedplayer')
squad_df = scrape_player_data(2022,2026,'teamsquad')

In [ ]:
final_df = pd.merge(
        squad_df,
        retained_df[['Player', 'Team', 'Year','Retained']],
        on=['Player', 'Team', 'Year'],
        how='left'
    )

final_df['Retained'] = final_df['Retained'].fillna('No')

In [ ]:
final_df['Price'] = final_df['Price'].astype(float)

In [ ]:
final_df.info()

In [ ]:
final_df[(final_df['Year']==2024) & (final_df['Team']=='CSK') & (final_df['Origin']=='Overseas')]

In [ ]:
final_df.to_csv('final.csv')

In [ ]:
df = pd.read_csv('IPL2022.csv')

In [ ]:
copy = final_df[final_df['Year']==2022].copy()

In [ ]:
copy

In [ ]:
df = df.rename(columns={'Name':'Player'})

In [ ]:
final22_df = pd.merge(
        copy,
        df[['Player', 'Year','C/U/A']],
        on=['Player', 'Year'],
        how='left'
    )

In [ ]:
final22_df.info()

In [ ]:
copy23 = final_df[final_df['Year']==2023].copy()

In [ ]:
copy23['Player'].to_csv('23Players.csv')

In [ ]:
copy23 = copy23[copy23['Player'] != 'Kuldip Yadav']

In [ ]:
copy23.reset_index(drop=True, inplace=True)

In [ ]:
df23 = pd.read_csv('2023players.csv')

In [ ]:
final23_df = pd.merge(
        copy23,
        df23[['Player', 'Year','C/U/A']],
        on=['Player', 'Year'],
        how='left'
    )

In [ ]:
na_rows = final23_df[final23_df['C/U/A'].isna()]

In [ ]:
na_rows

In [ ]:
final23_df=final23_df.fillna('Capped')

In [ ]:
final23_df.info()

In [ ]:
copy24 = final_df[final_df['Year']==2024].copy()

In [ ]:
copy24['Player'].to_csv('24Players.csv')

In [ ]:
df24 = pd.read_csv('2024Players.csv')

In [ ]:
final24_df = pd.merge(
        copy24,
        df24[['Player', 'Year','C/U/A']],
        on=['Player', 'Year'],
        how='left'
    )

In [ ]:
final24_df.info()

In [ ]:
final24_df.loc[(final24_df['Player'] == 'Amit Mishra') & (final24_df['C/U/A'].isna()), 'C/U/A'] = 'Capped'

In [ ]:
final24_df = final24_df.fillna('Uncapped')

In [ ]:
copy25 = final_df[final_df['Year']==2025]

In [ ]:
copy25['Player'].to_csv('25players.csv')

In [ ]:
df25 = pd.read_csv('2025Players.csv')

In [ ]:
final25_df = pd.merge(
        copy25,
        df25[['Player', 'Year','C/U/A']],
        on=['Player', 'Year'],
        how='left'
    )

In [ ]:
final25_df

In [ ]:
final25_df.info()

In [ ]:
super_final_auction = pd.concat([final22_df,final23_df,final24_df,final25_df])

In [ ]:
super_final_auction.to_csv('IPLAuctionData.csv')